In [1]:
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt

from extraction import extract_patches
from reconstruction import perform_voting

from model import Multimodel

Using TensorFlow backend.


In [3]:
import os
import nibabel as nib

from medical_data import cdr_info, nwbv_info, diff_info

orig_pattern = 'datasets/OASIS/OAS2_RAW_PART1/OAS2_00{0:02}_MR{1}/RAW/mpr-1_sstr_susan_matched.nii.gz'
prob_pattern = 'datasets/OASIS/OAS2_RAW_PART1/OAS2_00{0:02}_MR{1}/RAW/mpr-1_sstr_susan_pve_{2}.nii.gz'

step = (32, 32)
threshold = np.int32(0.30 * np.prod(curr_patch_shape[:]))
seg_train = np.empty((0, 1, ) + curr_patch_shape)
ref_train = np.empty((0, 2, ) + curr_patch_shape)
out_train = np.empty((0, 1, ) + curr_patch_shape)
for i in np.append(range(1, 76), range(80, 100)) :
    print '{} :'.format(i),
    
    for j in range(1, 6) :
        ref_filename = orig_pattern.format(i, j)

        if not os.path.exists(ref_filename) :
            continue
        
        volume_init = nib.load(ref_filename).get_data()

        mask_patches = extract_patches(volume_init != 0, (1, ) + curr_patch_shape, (1, ) + step)

        useful_patches = np.sum(mask_patches, axis=(1, 2, 3)) > threshold

        N = np.sum(useful_patches)

        volume_init = (volume_init - volume_init.mean()) / volume_init.std()

        ref_patches = extract_patches(volume_init, (1, ) + curr_patch_shape, (1, ) + step)
        ref_patches = ref_patches[useful_patches].reshape((-1, 1, ) + curr_patch_shape)

        ref_prob_patches = np.empty((N, 1, ) + curr_patch_shape)
        for a in range(1) :
            prob_filename = prob_pattern.format(i, j, a)
            prob_mask_init = nib.load(prob_filename).get_data()
            prob_patches = extract_patches(prob_mask_init, (1, ) + curr_patch_shape, (1, ) + step)
            prob_patches = prob_patches[useful_patches].reshape((-1, ) + curr_patch_shape)

            ref_prob_patches[:, a] = prob_patches

            del prob_patches
        
        diff = 0
        for k in range(j+1, 6) :
            mov_filename = orig_pattern.format(i, k)

            if not os.path.exists(mov_filename) :
                continue
    
            diff = diff + diff_info['OAS2_00{0:02}_MR{1}'.format(i, k)]
            if diff == 0 :
                continue

            volume_init = nib.load(mov_filename).get_data()
            volume_init = (volume_init - volume_init.mean()) / volume_init.std()

            mov_patches = extract_patches(volume_init, (1, ) + curr_patch_shape, (1, ) + step)
            mov_patches = mov_patches[useful_patches].reshape((-1, 1, ) + curr_patch_shape)

            ref_train = np.vstack((np.hstack((ref_patches, ref_prob_patches)), ref_train)) ##
            out_train = np.vstack((mov_patches, out_train)) ##
            del volume_init, mov_patches

            mov_prob_patches = np.empty((N, 1, ) + curr_patch_shape)
            for a in range(1) :
                prob_filename = prob_pattern.format(i, k, a)
                prob_mask_init = nib.load(prob_filename).get_data()
                prob_patches = extract_patches(prob_mask_init, (1, ) + curr_patch_shape, (1, ) + step)
                prob_patches = prob_patches[useful_patches].reshape((-1, ) + curr_patch_shape)

                mov_prob_patches[:, a] = prob_patches

                del prob_patches

            seg_train = np.vstack((mov_prob_patches, seg_train)) ##
            print '{}->{}'.format(j, k),
            del mov_prob_patches
        del ref_patches, ref_prob_patches
    print

1 : 1->2
2 : 1->2 1->3 2->3
3 :
4 : 1->2
5 : 1->3 2->3
6 :
7 : 1->3 1->4 3->4
8 : 1->2
9 : 1->2
10 : 1->2
11 :
12 : 1->2 1->3 2->3
13 : 1->3 2->3
14 :
15 :
16 : 1->2
17 : 1->3 1->4 1->5 3->5 4->5
18 : 1->4 3->4
19 :
20 : 1->2 1->3 2->3
21 : 1->2
22 : 1->2
23 : 1->2
24 :
25 :
26 : 1->2
27 : 1->3 1->4 2->3 2->4
28 : 1->2
29 :
30 :
31 : 1->3 2->3
32 :
33 :
34 : 1->4 2->4 3->4
35 :
36 : 1->3 1->4 1->5 3->4 3->5 4->5
37 : 1->2 1->3 1->4 2->3 2->4
38 :
39 : 1->2
40 : 1->3 2->3
41 : 1->3 2->3
42 : 1->2
43 : 1->2
44 : 1->2 1->3
45 : 1->2
46 :
47 :
48 : 1->2 1->3 1->4 1->5 2->3 2->4 2->5 3->4 3->5 4->5
49 :
50 : 1->2
51 : 1->2 1->3 2->3
52 : 1->2
53 : 1->2
54 : 1->2
55 : 1->2
56 : 1->2
57 : 1->3 2->3
58 : 1->3 2->3
59 :
60 : 1->2
61 : 1->2 1->3 2->3
62 : 1->2 1->3 2->3
63 : 1->2
64 : 1->3 2->3
65 :
66 :
67 : 1->2 1->3 1->4 2->3 2->4
68 : 1->2
69 :
70 : 1->2 1->3 1->4 1->5 2->3 2->4 2->5 3->4 3->5
71 : 1->2
72 :
73 : 1->2 1->3 1->4 1->5 2->3 2->4 2->5 3->4 3->5 4->5
74 :
75 :
80 : 1->2 1->3 2->3

In [6]:
curr_patch_shape = (128, 128)
input_modalities = ['T1', 'seg']
output_modalities = ['Gen']
output_weights = {'Gen' : 1.0, 'concat' : 1.0}
latent_dim = 32
channels = [2, 1]
patch_shape = curr_patch_shape
scale = 1
a_model = Multimodel(
    input_modalities, output_modalities, output_weights, latent_dim, channels, patch_shape, scale)
a_model.build()

Latent dimensions: 32
Fuse latent representations using max
making output: Tensor("enc_T1_act9_2/add:0", shape=(?, 32, 128, 128), dtype=float32) Tensor("dec_Gen_6/dec_Gen_act5/add:0", shape=(?, 1, 128, 128), dtype=float32) em_0_dec_Gen
making output: Tensor("enc_seg_act9_2/add:0", shape=(?, 32, 128, 128), dtype=float32) Tensor("dec_Gen_7/dec_Gen_act5/add:0", shape=(?, 1, 128, 128), dtype=float32) em_1_dec_Gen
making output: Tensor("combined_em_2/Maximum:0", shape=(?, 32, 128, 128), dtype=float32) Tensor("dec_Gen_8/dec_Gen_act5/add:0", shape=(?, 1, 128, 128), dtype=float32) em_2_dec_Gen
making output: em_concat Tensor("em_concat_2/concat:0", shape=(?, 2, ?), dtype=float32) em_concat_2/concat:0
all outputs:  [u'em_0_dec_Gen_2/add:0', u'em_1_dec_Gen_2/add:0', u'em_2_dec_Gen_2/add:0', u'em_concat_2/concat:0', u'em_fused_2/ExpandDims:0']
output dict:  {'em_concat': <function embedding_distance at 0x7fd94058e758>, 'em_fused': <function embedding_distance at 0x7fd94058e758>, 'em_1_dec_Gen': <

In [7]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

patience = 3

stopper = EarlyStopping(patience=patience)
checkpointer = ModelCheckpoint('model.h5', save_best_only=True, save_weights_only=True)

N = len(ref_train)
a_model.model.fit(
    [ref_train, seg_train],
    [out_train, out_train, out_train, np.empty((N, 2, 0)), np.empty((N, 1, 0))],
    validation_split=0.3, epochs=20,
    callbacks=[checkpointer, stopper])

Train on 38243 samples, validate on 16391 samples
Epoch 1/20
38243/38243 [==============================] - 573s 15ms/step - loss: 0.4244 - em_0_dec_Gen_loss: 0.1274 - em_1_dec_Gen_loss: 0.1879 - em_2_dec_Gen_loss: 0.1039 - em_concat_loss: 0.0052 - em_fused_loss: 0.0000e+00 - val_loss: 0.4064 - val_em_0_dec_Gen_loss: 0.1524 - val_em_1_dec_Gen_loss: 0.1489 - val_em_2_dec_Gen_loss: 0.1030 - val_em_concat_loss: 0.0021 - val_em_fused_loss: 0.0000e+00
Epoch 2/20
38243/38243 [==============================] - 567s 15ms/step - loss: 0.2835 - em_0_dec_Gen_loss: 0.1045 - em_1_dec_Gen_loss: 0.1046 - em_2_dec_Gen_loss: 0.0728 - em_concat_loss: 0.0015 - em_fused_loss: 0.0000e+00 - val_loss: 0.3981 - val_em_0_dec_Gen_loss: 0.1541 - val_em_1_dec_Gen_loss: 0.1440 - val_em_2_dec_Gen_loss: 0.0987 - val_em_concat_loss: 0.0013 - val_em_fused_loss: 0.0000e+00
Epoch 3/20
38243/38243 [==============================] - 567s 15ms/step - loss: 0.2550 - em_0_dec_Gen_loss: 0.0956 - em_1_dec_Gen_loss: 0.0931 - em

In [8]:
a_model.model.load_weights('model.h5')

In [9]:
from keras.models import Model
import os
import nibabel as nib
import SimpleITK as sitk

orig_pattern = 'datasets/OASIS/OAS2_RAW_PART1/OAS2_00{0:02}_MR{1}/RAW/mpr-1_sstr_susan_matched.nii.gz'
prob_pattern = 'datasets/OASIS/OAS2_RAW_PART1/OAS2_00{0:02}_MR{1}/RAW/mpr-1_sstr_susan_pve_{2}.nii.gz'

ref = 1
mov = 3
step = (32, 32)
for i in range(78, 79) :
    ref_filename = orig_pattern.format(i, ref)
    mov_filename = orig_pattern.format(i, mov)
    if not (os.path.exists(ref_filename) and os.path.exists(mov_filename)) :
        continue
    
    print i

    volume_data = nib.load(ref_filename)
    volume_actual = volume_data.get_data()
    volume_actual = (volume_actual - volume_actual.mean()) / volume_actual.std()
    ref_patches = extract_patches(volume_actual, (1, ) + curr_patch_shape, (1, ) + step)
    ref_patches = ref_patches.reshape((-1, 1, ) + curr_patch_shape)
    
    N = len(ref_patches)

    old_patches = np.empty((N, 1, ) + curr_patch_shape)
    for k in range(1) :
        filename = prob_pattern.format(i, ref, k)
        prob_mask_init = nib.load(filename).get_data()
        prob_patches = extract_patches(prob_mask_init, (1, ) + curr_patch_shape, (1, ) + step)
        prob_patches = prob_patches.reshape((-1, ) + curr_patch_shape)

        old_patches[:, k] = prob_patches

        del prob_patches
    
    new_patches = np.empty((N, 1, ) + curr_patch_shape)
    for k in range(1) :
        filename = prob_pattern.format(i, mov, k)
        prob_mask_init = nib.load(filename).get_data()
        prob_patches = extract_patches(prob_mask_init, (1, ) + curr_patch_shape, (1, ) + step)
        prob_patches = prob_patches.reshape((-1, ) + curr_patch_shape)

        new_patches[:, k] = prob_patches

        del prob_patches
    
    pred = a_model.model.predict([np.hstack((ref_patches, old_patches)), new_patches], verbose=1)[2]

    volume = perform_voting(pred.reshape((-1, 1, ) + curr_patch_shape), (1, ) + curr_patch_shape, (256, 256, 128), (1, ) + step)
    
    volume_data = nib.load(mov_filename)
    volume_actual = volume_data.get_data()
    volume = volume * volume_actual.std() + volume_actual.mean()
    
    volume = np.multiply(volume_data.get_data() != 0, volume)

    nib.save(nib.Nifti1Image(volume, volume_data.affine), 'result_{}_{}_{}.nii.gz'.format(i, ref, mov))
    
    del new_patches, ref_patches
    
    res = sitk.ReadImage('result_{}_{}_{}.nii.gz'.format(i, ref, mov))

    caster = sitk.CastImageFilter()
    caster.SetOutputPixelType(res.GetPixelID())

    orig = caster.Execute(sitk.ReadImage(orig_pattern.format(i, ref)))

    matcher = sitk.HistogramMatchingImageFilter()
    matcher.SetNumberOfHistogramLevels(1024)
    matcher.SetNumberOfMatchPoints(15)
    matched = matcher.Execute(res, orig)

    sitk.WriteImage(matched, 'result_cor_{}_{}_{}.nii.gz'.format(i, ref, mov))

78
1280/1280 [==============================] - 11s 8ms/step


In [ ]:
from keras.models import Model
import os
import nibabel as nib
import SimpleITK as sitk

orig_pattern = 'datasets/OASIS/OAS2_RAW_PART1/OAS2_00{0:02}_MR{1}/RAW/mpr-1_sstr_susan_matched.nii.gz'
prob_pattern = 'datasets/OASIS/OAS2_RAW_PART1/OAS2_00{0:02}_MR{1}/RAW/mpr-1_sstr_susan_pve_{2}.nii.gz'

ref = 1
mov = 3
step = (32, 32)
for i in range(78, 79) :
    ref_filename = orig_pattern.format(i, ref)
    mov_filename = orig_pattern.format(i, mov)
    if not (os.path.exists(ref_filename) and os.path.exists(mov_filename)) :
        continue
    
    print i

    volume_data = nib.load(ref_filename)
    volume_actual = volume_data.get_data()
    volume_actual = (volume_actual - volume_actual.mean()) / volume_actual.std()
    ref_patches = extract_patches(volume_actual, (1, ) + curr_patch_shape, (1, ) + step)
    ref_patches = ref_patches.reshape((-1, 1, ) + curr_patch_shape)
    
    N = len(ref_patches)

    old_patches = np.empty((N, 1, ) + curr_patch_shape)
    for k in range(1) :
        filename = prob_pattern.format(i, ref, k)
        prob_mask_init = nib.load(filename).get_data()
        prob_patches = extract_patches(prob_mask_init, (1, ) + curr_patch_shape, (1, ) + step)
        prob_patches = prob_patches.reshape((-1, ) + curr_patch_shape)

        old_patches[:, k] = prob_patches

        del prob_patches
    
    new_patches = np.empty((N, 1, ) + curr_patch_shape)
    for k in range(1) :
        filename = prob_pattern.format(i, mov, k)
        prob_mask_init = nib.load(filename).get_data()
        prob_patches = extract_patches(prob_mask_init, (1, ) + curr_patch_shape, (1, ) + step)
        prob_patches = prob_patches.reshape((-1, ) + curr_patch_shape)

        new_patches[:, k] = prob_patches

        del prob_patches
    
    z = a_model.predict_z(input_modalities, [np.hstack((ref_patches, old_patches)), new_patches])[2]
    
    for z_i in range(z.shape[1]) :
        volume = perform_voting(z[:, z_i, :, :].reshape((-1, 1, ) + curr_patch_shape), (1, ) + curr_patch_shape, (256, 256, 128), (1, ) + step)

        volume_data = nib.load(mov_filename)
        volume_actual = volume_data.get_data()
        volume = volume * volume_actual.std() + volume_actual.mean()

        volume = np.multiply(volume_data.get_data() != 0, volume)

        nib.save(nib.Nifti1Image(volume, volume_data.affine), 'result_{}_{}_{}_{}.nii.gz'.format(i, ref, mov, z_i))

    del new_patches, ref_patches